<a href="https://colab.research.google.com/github/adityashah841/Feature-engineering-and-parallelization-for-polynomial-regression-models-using-CART/blob/main/Modified_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [94]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None, var_red=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [65]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


    # def filter_dataset(self, tree, dataset):
    #     ''' function to filter dataset using decision tree '''
        
    #     if tree.value is not None:
    #         return dataset
    #     else:
    #         feature_index = tree.feature_index
    #         threshold = tree.threshold
            
    #         left_tree = tree.left
    #         right_tree = tree.right
            
    #         dataset_left = dataset[dataset[:, feature_index] <= threshold]
    #         dataset_right = dataset[dataset[:, feature_index] > threshold]
            
    #         filtered_dataset_left = self.filter_dataset(left_tree, dataset_left)
    #         filtered_dataset_right = self.filter_dataset(right_tree, dataset_right)
            
    #         return np.vstack((filtered_dataset_left, filtered_dataset_right))  

    def filter_dataset(self, dataset):
        ''' function to filter the dataset using the decisions of the decision tree '''
        
        subsets = []
        
        def filter_recursive(node, dataset):
            ''' recursive function to filter the dataset '''
            
            if node.value is not None:
                # add subset to the list
                subsets.append(dataset)
                return
            
            # split the dataset based on the decision
            dataset_left = dataset[dataset[:, node.feature_index] <= node.threshold]
            dataset_right = dataset[dataset[:, node.feature_index] > node.threshold]
            
            # recur left and right
            filter_recursive(node.left, dataset_left)
            filter_recursive(node.right, dataset_right)
        
        # call the recursive function
        filter_recursive(self.root, dataset)
        
        return subsets

In [66]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

    def filter_dataset(self, dataset):
        ''' function to filter the dataset using the decisions of the decision tree '''
        
        subsets = []
        
        def filter_recursive(node, dataset):
            ''' recursive function to filter the dataset '''
            
            if node.value is not None:
                # add subset to the list
                subsets.append(dataset)
                return
            
            # split the dataset based on the decision
            dataset_left = dataset[dataset[:, node.feature_index] <= node.threshold]
            dataset_right = dataset[dataset[:, node.feature_index] > node.threshold]
            
            # recur left and right
            filter_recursive(node.left, dataset_left)
            filter_recursive(node.right, dataset_right)
        
        # call the recursive function
        filter_recursive(self.root, dataset)
        
        return subsets        

In [23]:
!unzip ./gdrive/MyDrive/ModifiedDecisionTree/AirBnB\ prices.zip

Archive:  ./gdrive/MyDrive/ModifiedDecisionTree/AirBnB prices.zip
   creating: AirBnB prices/
  inflating: __MACOSX/._AirBnB prices  
  inflating: AirBnB prices/amsterdam_weekdays.csv  
  inflating: __MACOSX/AirBnB prices/._amsterdam_weekdays.csv  
  inflating: AirBnB prices/barcelona_weekends.csv  
  inflating: __MACOSX/AirBnB prices/._barcelona_weekends.csv  
  inflating: AirBnB prices/rome_weekdays.csv  
  inflating: __MACOSX/AirBnB prices/._rome_weekdays.csv  
  inflating: AirBnB prices/berlin_weekdays.csv  
  inflating: __MACOSX/AirBnB prices/._berlin_weekdays.csv  
  inflating: AirBnB prices/budapest_weekdays.csv  
  inflating: __MACOSX/AirBnB prices/._budapest_weekdays.csv  
  inflating: AirBnB prices/vienna_weekdays.csv  
  inflating: __MACOSX/AirBnB prices/._vienna_weekdays.csv  
  inflating: AirBnB prices/london_weekdays.csv  
  inflating: __MACOSX/AirBnB prices/._london_weekdays.csv  
  inflating: AirBnB prices/lisbon_weekends.csv  
  inflating: __MACOSX/AirBnB prices/._lisb

In [24]:
import os

In [25]:
len(os.listdir('./AirBnB prices'))

20

In [26]:
dataset = pd.read_csv('./AirBnB prices/amsterdam_weekdays.csv')
dataset = dataset.drop(dataset.index[:])
dataset['location'] = []

In [27]:
dataset

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,...,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,location


In [28]:
for file in os.listdir('./AirBnB prices'):
  if file.endswith('weekdays.csv'):
    new_data = pd.read_csv('./AirBnB prices/'+file)
    new_data['location'] = file.split('_')[0]
    dataset = pd.concat([dataset, new_data], ignore_index=True)

In [29]:
dataset['location'].unique()

array(['amsterdam', 'vienna', 'london', 'lisbon', 'paris', 'athens',
       'berlin', 'rome', 'barcelona', 'budapest'], dtype=object)

In [30]:
dataset.columns

Index(['Unnamed: 0', 'realSum', 'room_type', 'room_shared', 'room_private',
       'person_capacity', 'host_is_superhost', 'multi', 'biz',
       'cleanliness_rating', 'guest_satisfaction_overall', 'bedrooms', 'dist',
       'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index',
       'rest_index_norm', 'lng', 'lat', 'location'],
      dtype='object')

In [31]:
y = dataset["realSum"]

In [164]:
X = dataset.drop(dataset.columns[:1],axis=1)
full_dataset_headers = X

In [165]:
full_dataset_headers = pd.get_dummies(full_dataset_headers,columns=["room_type","room_shared","room_private","host_is_superhost","location"],drop_first=True).columns
full_dataset_headers

Index(['realSum', 'person_capacity', 'multi', 'biz', 'cleanliness_rating',
       'guest_satisfaction_overall', 'bedrooms', 'dist', 'metro_dist',
       'attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm', 'lng',
       'lat', 'room_type_Private room', 'room_type_Shared room',
       'room_shared_True', 'room_private_True', 'host_is_superhost_True',
       'location_athens', 'location_barcelona', 'location_berlin',
       'location_budapest', 'location_lisbon', 'location_london',
       'location_paris', 'location_rome', 'location_vienna'],
      dtype='object')

In [153]:
X.drop(["realSum"],axis=1,inplace=True)

In [154]:
X = pd.get_dummies(X,columns=["room_type","room_shared","room_private","host_is_superhost","location"],drop_first=True)

In [155]:
X

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,...,host_is_superhost_True,location_athens,location_barcelona,location_berlin,location_budapest,location_lisbon,location_london,location_paris,location_rome,location_vienna
0,2.0,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,...,0,0,0,0,0,0,0,0,0,0
1,4.0,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,...,0,0,0,0,0,0,0,0,0,0
2,2.0,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,...,0,0,0,0,0,0,0,0,0,0
3,4.0,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,...,0,0,0,0,0,0,0,0,0,0
4,2.0,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,4.0,1,0,9.0,86.0,1,0.873765,0.249505,211.862902,12.634381,...,0,0,0,0,1,0,0,0,0,0
25496,4.0,0,1,10.0,95.0,1,0.848468,0.227851,243.410062,14.515686,...,0,0,0,0,1,0,0,0,0,0
25497,2.0,0,0,10.0,100.0,0,2.661352,0.985169,100.821179,6.012441,...,0,0,0,0,1,0,0,0,0,0
25498,6.0,0,1,2.0,40.0,2,0.861198,0.478868,221.524602,13.210553,...,0,0,0,0,1,0,0,0,0,0


In [156]:
len(full_dataset_headers)

29

In [84]:
reg = DecisionTreeRegressor()

In [85]:
from sklearn.model_selection import train_test_split

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.99, random_state=42)

In [97]:
X_train.shape, y_train.shape

((255, 28), (255,))

In [98]:
y_train = y_train.to_numpy().reshape(-1,1)

In [99]:
reg.fit(X_train,y_train)

In [134]:
X_train.shape, y_train.shape

((255, 28), (255, 1))

In [100]:
subsets = reg.filter_dataset(np.concatenate((X_train,y_train),axis=1))

In [101]:
for subset in subsets:
  print(len(subset))

66
69
35
36
4
26
17
2


In [167]:
full_dataset_headers = list(full_dataset_headers)[1:]

In [168]:
full_dataset_headers.append('realSum')

In [ ]:
full_dataset_headers

In [170]:
pd.DataFrame(subsets[-1],columns = full_dataset_headers)

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,...,location_athens,location_barcelona,location_berlin,location_budapest,location_lisbon,location_london,location_paris,location_rome,location_vienna,realSum
0,3.0,0.0,0.0,10.0,98.0,2.0,0.691933,0.808129,1029.727093,54.524734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1429.240972
1,4.0,0.0,0.0,10.0,97.0,2.0,1.541236,0.440917,541.768981,37.658963,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1642.794986


In [53]:
import pickle

In [102]:
filehandler = open('model.pkl', 'wb') 
pickle.dump(reg, filehandler)

In [103]:
filehandler = open('./model.pkl', 'rb') 

In [104]:
model_loaded = pickle.load(filehandler)

In [105]:
model_loaded.print_tree()

X_5 <= 1.0 ? None
 left:X_12 <= 12.49646 ? None
  left:X_14 <= 0.0 ? None
    left:328.3619836406945
    right:214.1459560620793
  right:X_0 <= 2.0 ? None
    left:130.02508081075942
    right:184.37480105518432
 right:X_13 <= 48.21893 ? None
  left:X_6 <= 0.6188685601990687 ? None
    left:595.6621051646118
    right:269.91790209758307
  right:X_8 <= 460.1643084572228 ? None
    left:685.7385568136793
    right:1536.0179789071399
